# URBAG map checks

This notebook reads in the URBAG map(s) and makes initial checks of the total data it for nutrient requirements of crops, areas, food production, etc.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#In this cell the scenario is chosen. This can either be defined here or in the RUN_ALL.ipynb
#If defined here unmark the scen = line and select the scenario you will run.
#If defined in RUN_ALL.ipynb unmark the %store -r scen line.
#Scenario names are:
#S0_MinFert > N,P,K demand met by mineral fertilizers only
#S0_struvite_P > P demand met by struvite, N demand met by struvite and mineral fertilizer and K demand met by mineral fertilizer
#S0_compost > N,P,K supplied by compost produced in the AMB, remaining N,P,K demand met by mineral fertilizer
#S0_Ammon_salts > N supplied from recovered ammonium salts and from mineral fertilizers, P from struvite and K from mineral fertilizer
scen = 'S0_MinFert'
#%store -r scen

In [ ]:
print(scen)

In [ ]:
#This cell determines the path and name of the map file
filename = "S4_NewVoronoi_0524.csv"
file_path = f"inputs/{filename}"

In [ ]:
URBAG_map = pd.read_csv(file_path)

In [ ]:
URBAG_map.head()

In [ ]:
URBAG_map.info()

In [ ]:
URBAG_map.describe()

In [ ]:
#This cell determines the categories in the map
map = URBAG_map.keys().to_frame()
map

In [ ]:
#This cell determines the filename and path to save the categories of the map
filename = "labels_S4.xlsx"
directory = "output/"
file_path = f"{directory}{filename}"

In [ ]:
map.to_excel(file_path)

In [ ]:
len_URBAG_map = len(URBAG_map['Voronoi_1'])
print(f"This is the amount of plots in the map: {len_URBAG_map}")

In [ ]:
#How many crop categories
len_URBAG_map_crops_cat = len(pd.unique(URBAG_map["Voronoi_1"]))
print(f"This is the amount of crop categories in the map: {len_URBAG_map_crops_cat}")

Here we check that only plots in transformation or with fallow land do not get input of nutrients.
The rest are assumed to have a requirement of nutrients.

In [ ]:
#Check for no P fertilizer inputs
pd.unique(URBAG_map.loc[URBAG_map["kgP2O5/ha"]==0]["Voronoi_1"])

In [ ]:
#Check for no N inputs
pd.unique(URBAG_map.loc[URBAG_map["kgN/ha"]==0]["Voronoi_1"])

In [ ]:
#Check for no K inputs
pd.unique(URBAG_map.loc[URBAG_map["kgK2O/ha"]==0]["Voronoi_1"])

In [ ]:
data_N = {'Voronoi': URBAG_map.loc[URBAG_map["kgN/ha"]!=0]["Voronoi_1"], 'CNR_N': URBAG_map.loc[URBAG_map["kgN/ha"]!=0]["kgN/ha"]}
df_N = pd.DataFrame(data_N)

In [ ]:
data_P = {'Voronoi': URBAG_map.loc[URBAG_map["kgP2O5/ha"]!=0]["Voronoi_1"], 'CNR_P': URBAG_map.loc[URBAG_map["kgP2O5/ha"]!=0]["kgP2O5/ha"]}
df_P = pd.DataFrame(data_P)

In [ ]:
data_K = {'Voronoi': URBAG_map.loc[URBAG_map["kgK2O/ha"]!=0]["Voronoi_1"], 'CNR_K': URBAG_map.loc[URBAG_map["kgK2O/ha"]!=0]["kgK2O/ha"]}
df_K = pd.DataFrame(data_K)

In [ ]:
df_N.sort_values(by="Voronoi")

In [ ]:
df_P.sort_values(by="Voronoi")

In [ ]:
df_K.sort_values(by="Voronoi")

In [ ]:
#crop nutrient content for N,P,K in URBAG map saved to output folder
df_N.sort_values(by="Voronoi").drop_duplicates('Voronoi', keep='first').to_excel("output/CNR_N_FromInputMap.xlsx")

In [ ]:
df_P.sort_values(by="Voronoi").drop_duplicates('Voronoi', keep='first').to_excel("output/CNR_P_FromInputMap.xlsx")

In [ ]:
df_K.sort_values(by="Voronoi").drop_duplicates('Voronoi', keep='first').to_excel("output/CNR_K_FromInputMap.xlsx")

In [ ]:
#checking min and max for different crop categories
max(URBAG_map.loc[URBAG_map["Voronoi_1"]=="Fruiters no cítrics en regadiu"]["kgP2O5/ha"])

# Totals from map

In [ ]:
#Total food production in the AMB 68716.3 tons/yr > S0
#Total food production in the AMB 73673.54300666731 tons/yr > S4
URBAG_map['Production'] = (URBAG_map['Area']*(1/10000)) * (URBAG_map['yield_kgHA']/1000)
Total_food = np.sum(URBAG_map['Production'])
Total_food

In [ ]:
URBAG_map.keys()

In [ ]:
#Total area on ha (6984.425078) S4
#Total area in HA (5587.4) S0
#Total area in HA (5568.5343175) (NPK_27_06_22An - 3_02_23.xlsx)
#Total area in HA (5568.5343175) (NPK_27_06_22An - 24_02_23.xlsx)
#Total area in HA (5568.5343175) (NPK_27_06_22An_05_03_23.xlsx)
#Total area in HA (5568.5343175) (S0_clean.xlsx)
Total_area = np.sum(URBAG_map['Area'])*(1/10000)
Total_area

In [ ]:
#Total N input excluding N fixation 810.58229 tons N/yr
#Total N input excluding N fixation 769.06544476727 tons N/yr (NPK_27_06_22An - 24_02_23.xlsx)
#Total N input excluding N fixation 769.06544476727 tons N/yr (NPK_27_06_22An_05_03_23.xlsx)
#Total N input excluding N fixation 769.06544476727 tons N/yr (.xlsx)
#Total N input excluding N fixation 925.551945239332 tons N/yr (S4_NewVoronoi_0224.xlsx)
URBAG_map['N_kgyr'] = URBAG_map['kgN/ha'] * URBAG_map['Area']*(1/10000)
Total_N_require = np.sum(URBAG_map['N_kgyr'])/1000
Total_N_require

In [ ]:
#Total P input including 305.88348 tons P2O5/yr 
#tons P2O5/yr 346.4701826638308 (NPK_27_06_22An - 24_02_23.xlsx)
#tons P2O5/yr 232.1972074557924 (NPK_27_06_22An_05_03_23.xlsx)
#Total P input including 133.48755 tons P/yr
#tons P/yr 151.29702299730602 (NPK_27_06_22An - 24_02_23.xlsx)
#tons P/yr 101.39616046104473 (NPK_27_06_22An_05_03_23.xlsx)
#tons P/yr 101.39616046104473 (S0_NewVoronoi_0224.xlsx)
#tons P/yr 116.34397127690181 (S4_NewVoronoi_0224.xlsx)
URBAG_map['P2O5_kgyr'] = URBAG_map['kgP2O5/ha'] * URBAG_map['Area']*(1/10000)
URBAG_map['P_kgyr'] = URBAG_map['P2O5_kgyr']/2.29
Total_P2O5_require = np.sum(URBAG_map['P2O5_kgyr'])/1000
Total_P_require = np.sum(URBAG_map['P_kgyr'])/1000
Total_P_require

In [ ]:
#Total K input 710.56676 tons K2O/yr
#Total K input 586.500652283024 tons K/yr, 709.665789262459 tons K2O/yr (NPK_27_06_22An - 24_02_23.xlsx)
#Total K input 586.221985694181 tons K/yr, 709.328602689959 tons K2O/yr (NPK_27_06_22An_05_03_23.xlsx)
#Total K input 586.221985694181 tons K/yr, 709.328602689959 tons K2O/yr (S0_NewVoronoi_0224.xlsx)
#Total K input 586.221985694181 tons K/yr, 894.641297301327 tons K2O/yr (S4_NewVoronoi_0224.xlsx)
URBAG_map['K2O_kgyr'] = URBAG_map['kgK2O/ha'] * URBAG_map['Area']*(1/10000)
URBAG_map['K_kgyr'] = URBAG_map['K2O_kgyr']/1.21
Total_K2O_require = np.sum(URBAG_map['K2O_kgyr'])/1000
Total_K_require = np.sum(URBAG_map['K_kgyr'])/1000
Total_K2O_require

In [ ]:
#kg N/ha average for the whole AMB
N_HA_average_AMB = Total_N_require*1000/Total_area
N_HA_average_AMB

In [ ]:
#kg P/ha average for the whole AMB
P_HA_average_AMB = Total_P_require*1000/Total_area
P_HA_average_AMB

In [ ]:
#kg P/ha average for the whole AMB
P2O5_HA_average_AMB = Total_P2O5_require*1000/Total_area
P2O5_HA_average_AMB

In [ ]:
#kg K/ha average for the whole AMB
K_HA_average_AMB = Total_K_require*1000/Total_area
K_HA_average_AMB

In [ ]:
#kg K/ha average for the whole AMB
K2O_HA_average_AMB = Total_K2O_require*1000/Total_area
K2O_HA_average_AMB

In [ ]:
my_array = np.array([[Total_food,
                     Total_area,
                     Total_N_require,
                     Total_P_require,
                     Total_P2O5_require,
                     Total_K_require,
                     Total_K2O_require,
                     N_HA_average_AMB,
                     P_HA_average_AMB,
                     P2O5_HA_average_AMB,
                     K_HA_average_AMB,
                     K2O_HA_average_AMB]])

In [ ]:
Totals_Map = pd.DataFrame(my_array, columns = ['Food_tonsYR',
                                               'Area_HA',
                                               'Nitrogen_NtonsYR',
                                               'Phosphorus_PtonsYR',
                                               'Phosphorus_P2O5tonsYR',
                                               'Potassium_KtonsYR',
                                               'Potassium_K2OtonsYR',
                                               'Nitrogen_NkgHA',
                                               'Phosphorus_PkgHA',
                                               'Phosphorus_P2O5_kgHA',
                                               'Potassium_KkgHA',
                                               'Potassium_K2OkgHA'])

In [ ]:
Totals_AMB = Totals_Map.T
Totals_AMB

In [ ]:
Total_food_2 = np.sum(URBAG_map['Production'])
Total_food_2

In [ ]:
Total_N_require_2 = np.sum(URBAG_map['N_T'])
Total_N_require_2

In [ ]:
Total_P_require_2 = np.sum(URBAG_map['P_T'])
Total_P_require_2

In [ ]:
Total_K_require_2 = np.sum(URBAG_map['K_T'])
Total_K_require_2

Here we check that the totals found with the sum of total nutrient requirement in the map is the same as we find here:

In [ ]:
Total_food_2 == Total_food

In [ ]:
Total_N_require_2 == Total_N_require

In [ ]:
Total_N_require_2-Total_N_require

In [ ]:
Total_P_require_2 == Total_P2O5_require

In [ ]:
Total_P_require_2-Total_P2O5_require

In [ ]:
Total_K_require_2 == Total_K2O_require

In [ ]:
Total_K_require_2-Total_K2O_require

## Saving outputs

In [ ]:
#Variables for next notebook
%store Totals_AMB
%store Total_N_require
%store Total_P2O5_require
%store Total_K2O_require

In [ ]:
#Total for AMB from maps without modifications
Totals_AMB.to_excel("output/" + scen + "/totals_FromInputMap.xlsx")